In [1]:
# Data Cleaning
import pandas as pd
df = pd.read_csv('bidding_data.csv')
group_cols = [col for col in df.columns if col not in ['bid_made', 'alone', 'score', 'hand_id']]
df_filtered = df.groupby(group_cols).filter(lambda g: len(g) > 2)
grouped_data = df_filtered.groupby(group_cols).apply(
    lambda x: pd.Series({
        'bid_win_rate': (x.loc[x['bid_made'] == 1, 'score']>0).mean(),
        'bid_expected_score': x.loc[x['bid_made'] == 1, 'score'].mean(),
        'not_bid_win_rate': (x.loc[x['bid_made'] == 0, 'score']>0).mean(),
        'not_bid_expected_score': x.loc[x['bid_made'] == 0, 'score'].mean(),
        'alone_win_rate': (x.loc[x['alone'] == 1, 'score']>0).mean(),
        'alone_expected_score': x.loc[x['alone'] == 1, 'score'].mean()
    })
)
grouped_data.to_csv("grouped.csv")
mask = ~(
    (

        
        grouped_data["bid_expected_score"].isna() & grouped_data["not_bid_expected_score"].between(-1, 1)
    ) | (
        grouped_data["not_bid_expected_score"].isna() & grouped_data["bid_expected_score"].between(-1, 1)
    )
)

# Keep rows that do NOT satisfy the condition
grouped_data_cleaned = grouped_data[mask]

def should_bid(row):
    a, b = row["bid_expected_score"], row["not_bid_expected_score"]
    
    # Case 1: both exist
    if pd.notna(a) and pd.notna(b):
        return int(a > b)
    
    # Case 2: only a exists
    if pd.notna(a) and pd.isna(b):
        if a > 1:
            return 1
        elif a < -1:
            return 0
    
    # Case 3: only b exists
    if pd.isna(a) and pd.notna(b):
        if b < -1:
            return 1
        elif b > 1:
            return 0
grouped_data_cleaned["should_bid"] = grouped_data_cleaned.apply(should_bid, axis=1)

def should_alone(row):
    a, b = row["bid_expected_score"], row["alone_expected_score"]

    if pd.notna(b) and (b > 1.5 or b > a):
        return 1

    else:
        return 0
grouped_data_cleaned["should_alone"] = grouped_data_cleaned.apply(should_alone, axis=1)

grouped_data_cleaned.to_csv("grouped_cleaned.csv")

/var/folders/34/z3yh7c2n4zd4rh6q0kqkfbfm0000gn/T/ipykernel_13838/2336717772.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = df_filtered.groupby(group_cols).apply(
/var/folders/34/z3yh7c2n4zd4rh6q0kqkfbfm0000gn/T/ipykernel_13838/2336717772.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_data_cleaned["should_bid"] = grouped_data_cleaned.apply(should_bid, axis=1)
/var/folders/34/z3yh7c2n4zd4rh6q0kqkfbfm0000gn/T/ipykernel_13838/2

In [2]:
# Model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Load data
df = pd.read_csv('grouped_cleaned.csv')

# Define features and targets
features = ['num_trump', 'right_bower', 'left_bower', 'trump_ace', 'trump_king', 
            'trump_queen', 'trump_ten', 'trump_nine', 'offsuit_aces', 'flipped_jack', 
            'flipped_ace', 'seat_position', 'bidding_round']
X = df[features]
y_bid = df['should_bid']
y_alone = df['should_alone']

# Split data (same splits for both targets)
X_train, X_test, _, _ = train_test_split(X, y_bid, test_size=0.2, random_state=42)
_, _, y_bid_train, y_bid_test = train_test_split(X, y_bid, test_size=0.2, random_state=42)
_, _, y_alone_train, y_alone_test = train_test_split(X, y_alone, test_size=0.2, random_state=42)

# Model Pipelines ---------------------------------------------------

# 1. BIDDING MODELS
bid_logreg = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000, random_state=42))
])

bid_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=42))
])

# 2. GOING ALONE MODELS
alone_logreg = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000, random_state=42))
])

alone_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=42))
])

# Training ----------------------------------------------------------

# Train bidding models
bid_logreg.fit(X_train, y_bid_train)
bid_rf.fit(X_train, y_bid_train)

# Train going-alone models
alone_logreg.fit(X_train, y_alone_train)
alone_rf.fit(X_train, y_alone_train)

# Evaluation --------------------------------------------------------

def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    print(f"\n{model_name} Performance:")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    return model

# Evaluate all models
bid_logreg = evaluate_model(bid_logreg, X_test, y_bid_test, "Bidding Logistic Regression")
bid_rf = evaluate_model(bid_rf, X_test, y_bid_test, "Bidding Random Forest")

alone_logreg = evaluate_model(alone_logreg, X_test, y_alone_test, "Going Alone Logistic Regression")
alone_rf = evaluate_model(alone_rf, X_test, y_alone_test, "Going Alone Random Forest")

# Prediction Example ------------------------------------------------

def make_prediction(model, input_data, model_name):
    sample_input = pd.DataFrame([input_data], columns=features)
    prediction = model.predict(sample_input)
    proba = model.predict_proba(sample_input)
    print(f"\n{model_name} Prediction:")
    print(f"Predicted class: {prediction[0]}")
    print(f"Probability estimates: {proba[0]}")
    return prediction

# Sample input: [num_trump, right_bower, left_bower, trump_ace, trump_king, 
#                trump_queen, trump_ten, trump_nine, offsuit_aces, flipped_jack, 
#                flipped_ace, seat_position, bidding_round]
sample_hand = [5, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0]

make_prediction(bid_logreg, sample_hand, "Bidding Logistic Regression")
make_prediction(bid_rf, sample_hand, "Bidding Random Forest")
make_prediction(alone_logreg, sample_hand, "Going Alone Logistic Regression")
make_prediction(alone_rf, sample_hand, "Going Alone Random Forest")

# Feature Importance ------------------------------------------------

print("\nBidding Random Forest Feature Importance:")
print(pd.DataFrame({
    'Feature': features,
    'Importance': bid_rf.named_steps['rf'].feature_importances_
}).sort_values('Importance', ascending=False))

print("\nGoing Alone Random Forest Feature Importance:")
print(pd.DataFrame({
    'Feature': features,
    'Importance': alone_rf.named_steps['rf'].feature_importances_
}).sort_values('Importance', ascending=False))


Bidding Logistic Regression Performance:
              precision    recall  f1-score   support

           0       0.83      0.84      0.83       203
           1       0.72      0.69      0.71       118

    accuracy                           0.79       321
   macro avg       0.77      0.77      0.77       321
weighted avg       0.79      0.79      0.79       321

Accuracy: 0.7882

Bidding Random Forest Performance:
              precision    recall  f1-score   support

           0       0.84      0.88      0.86       203
           1       0.77      0.71      0.74       118

    accuracy                           0.82       321
   macro avg       0.81      0.79      0.80       321
weighted avg       0.81      0.82      0.81       321

Accuracy: 0.8162

Going Alone Logistic Regression Performance:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       258
           1       0.75      0.75      0.75        63

    accuracy            

In [ ]:
# Test
from importlib import reload
from euchre import play_euchre

%reload_ext autoreload
%autoreload 2

winning_score = 2000

play_euchre(winning_score, player_strategies = ['new', 'scorecard_complex', 'new', 'scorecard_complex'], verbose=True)


----------------------------------------------------------------------------------------------------
Dealer: Player1
Player0: A♣ J♥ K♥ J♣ T♠
Player1: Q♣ 9♣ K♦ 9♥ 9♦
Player2: K♠ T♣ J♦ K♣ T♥
Player3: Q♦ A♥ T♦ Q♥ A♠

Flipped: 9♠
Player2 passes
Player3 passes
Player0 passes
Player1 passes
Player2 bids ♥ 

Player0: J♥ K♥ A♣ J♣ T♠
Player1: 9♥ K♦ Q♣ 9♣ 9♦
Player2: J♦ T♥ K♠ K♣ T♣
Player3: A♥ Q♥ A♠ Q♦ T♦

Player2 leads: J♦ Q♥ K♥ 9♥ Player2 takes (1-0)
Player2 leads: K♣ A♥ J♣ 9♣ Player3 takes (1-1)
Player3 leads: A♠ T♠ 9♦ K♠ Player3 takes (1-2)
Player3 leads: Q♦ J♥ K♦ T♣ Player0 takes (2-2)
Player0 leads: A♣ Q♣ T♥ T♦ Player2 takes (3-2)

Team0 wins the round (1-0)
----------------------------------------------------------------------------------------------------
Dealer: Player2
Player0: A♦ Q♦ A♣ 9♥ 9♠
Player1: 9♦ T♦ Q♥ K♦ Q♣
Player2: K♥ T♠ J♦ T♥ K♣
Player3: A♠ J♠ K♠ 9♣ J♣

Flipped: T♣
Player3 bids ♣ ALONE

Player0: A♣ A♦ Q♦ 9♥ 9♠
Player1: Q♣ K♦ Q♥ T♦ 9♦
Player2: K♣ T♣ K♥ J♦ T♥
Player3: J♣ J♠ 9♣

In [ ]:
# Test
from importlib import reload
from euchre import play_euchre

%reload_ext autoreload
%autoreload 2

winning_score = 2000

play_euchre(winning_score, player_strategies = ['new', 'scorecard_simple', 'new', 'scorecard_simple'], verbose=True)


----------------------------------------------------------------------------------------------------
Dealer: Player2
Player0: A♠ T♥ T♦ K♠ J♦
Player1: 9♠ T♠ Q♠ 9♦ A♦
Player2: A♣ J♠ K♣ Q♦ 9♥
Player3: Q♥ A♥ T♣ K♥ J♣

Flipped: K♦
Player3 passes
Player0 passes
Player1 passes
Player2 passes
Player3 bids ♣ 

Player0: A♠ K♠ J♦ T♥ T♦
Player1: A♦ Q♠ T♠ 9♠ 9♦
Player2: J♠ A♣ K♣ Q♦ 9♥
Player3: J♣ T♣ A♥ K♥ Q♥

Player3 leads: J♣ T♥ 9♠ K♣ Player3 takes (0-1)
Player3 leads: A♥ T♦ 9♦ 9♥ Player3 takes (0-2)
Player3 leads: K♥ J♦ T♠ A♣ Player2 takes (1-2)
Player2 leads: J♠ T♣ K♠ Q♠ Player2 takes (2-2)
Player2 leads: Q♦ Q♥ A♠ A♦ Player1 takes (2-3)

Team1 wins the round (0-1)
----------------------------------------------------------------------------------------------------
Dealer: Player3
Player0: J♠ Q♥ J♣ T♣ Q♣
Player1: A♥ A♦ 9♠ A♠ Q♠
Player2: T♥ K♦ Q♦ K♣ K♠
Player3: T♠ A♣ 9♣ K♥ T♦

Flipped: J♦
Player0 passes
Player1 passes
Player2 passes
Player3 bids ♦ 

Player0: Q♥ Q♣ J♠ J♣ T♣
Player1: A♦ A♥ A♠ Q♠ 9♠
P